In [ ]:
!pip install tensorflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
from scipy.io import wavfile as wav
from tqdm import tqdm
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
A LINK TO THE DATA SET WHICH IS SHARED WITH ANYONE
https://drive.google.com/drive/folders/1HyRuOPT4w6yLwshzmbs8QkCGB4OkFGjr?usp=sharing
"""

'\nA LINK TO THE DATA SET WHICH IS SHARED WITH ANYONE\nhttps://drive.google.com/drive/folders/1HyRuOPT4w6yLwshzmbs8QkCGB4OkFGjr?usp=sharing\n'

In [ ]:
"""
Getting my data set from a CSV file. Then placing it into Pandas.
"""
data_set =pd.read_csv(r'/content/drive/MyDrive/BigDataProject1/Sounds8K/metadata/Sound2.csv')
data_set.head(10)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,0,local_sound
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,0,local_sound
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,0,local_sound
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,0,local_sound
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,0,local_sound
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,0,local_sound
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,0,local_sound
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,0,local_sound
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,horn


In [ ]:
"""
The classes I will be working with are Local Sounds, Construction, Barking, Siren, horn and gun sounds.
These are the classes receieved from the CSV file.
"""
data_set['class'].value_counts()

local_sound     3000
construction    3000
bark            1000
siren            929
horn             429
gun              374
Name: class, dtype: int64

In [ ]:
"""
Here is an example of what I did to WAV file, to which I will do to all WAV files in the data set.
"""
audio_file_path='/content/drive/MyDrive/BigDataProject1/Sounds8K/local.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [ ]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)

In [ ]:
"""
The shape of this WAV file is 40 Rows and 173 columbs.
"""
print(mfccs.shape)

(40, 173)


In [ ]:
mfccs

array([[-4.45197296e+02, -4.47219299e+02, -4.49755127e+02, ...,
        -4.77412781e+02, -4.74241730e+02, -4.82704987e+02],
       [ 1.12513969e+02,  1.11970390e+02,  1.12244164e+02, ...,
         1.12045395e+02,  1.12248581e+02,  1.05560913e+02],
       [-1.58260937e+01, -2.30021858e+01, -3.12500191e+01, ...,
        -9.15441895e+00, -1.03232269e+01, -7.39410734e+00],
       ...,
       [-7.82766438e+00, -5.03880644e+00, -4.48165131e+00, ...,
        -1.90693259e-01,  4.34143877e+00,  1.00339069e+01],
       [-1.91763473e+00, -8.02736938e-01, -1.20930755e+00, ...,
        -1.23640180e-01,  2.90505886e-02,  9.22017217e-01],
       [-3.88129652e-01,  3.09316814e-01,  6.72155762e+00, ...,
        -2.33736706e+00, -4.25179911e+00, -2.31322789e+00]], dtype=float32)

In [ ]:
"""
Getting the path to folder of the data set.
"""
audio_path="/content/drive/MyDrive/BigDataProject1/Sounds8K/audio"

In [ ]:
"""
I created a definition to do what I did above, getting info of the data in the audio file.

"""
def wav_features(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    wav_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_wav_features = np.mean(wav_features.T,axis=0)
    return mfccs_wav_features

In [ ]:
"""
Using the above definition, the data will be extract from the data set.
"""
extracted_wav_data=[]
for index_num,row in tqdm(data_set.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=wav_features(file_name)
    extracted_wav_data.append([data,final_class_labels])

3555it [19:46,  3.08it/s]/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1323
  n_fft, y.shape[-1]
8326it [48:32,  3.49it/s]/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1103
  n_fft, y.shape[-1]
8329it [48:33,  3.22it/s]/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1523
  n_fft, y.shape[-1]
8732it [50:54,  2.86it/s]


In [ ]:
"""
Taking the data extracted from the data set and placing it into a Pandas framework.
"""
extracted_wav_data_df=pd.DataFrame(extracted_wav_data,columns=['feature','class'])
extracted_wav_data_df.head()

,feature,class
0,"[-215.79301, 71.66612, -131.81377, -52.09133, ...",bark
1,"[-424.68677, 110.56227, -54.148235, 62.01074, ...",local_sound
2,"[-459.56467, 122.800354, -47.92471, 53.265705,...",local_sound
3,"[-414.55377, 102.896904, -36.66495, 54.18041, ...",local_sound
4,"[-447.397, 115.0954, -53.809113, 61.60859, 1.6...",local_sound


In [ ]:
"""
Defining the two variables to be used to corelates the class with the relavint data.
"""
data_feature=np.array(extracted_wav_data_df['feature'].tolist())
data_class=np.array(extracted_wav_data_df['class'].tolist())

In [ ]:
data_feature.shape

(8732, 40)

In [ ]:
data_feature
# normalise, check min and max divide by max, checkin if impacting on resutls.

array([[-2.15793015e+02,  7.16661224e+01, -1.31813766e+02, ...,
        -1.68275905e+00, -8.85858238e-01,  3.54388624e-01],
       [-4.24686768e+02,  1.10562271e+02, -5.41482353e+01, ...,
         6.17408633e-01, -6.84974074e-01,  5.71514428e-01],
       [-4.59564667e+02,  1.22800354e+02, -4.79247093e+01, ...,
         2.22541404e+00,  1.54661798e+00, -8.36315691e-01],
       ...,
       [-3.04613159e+02,  1.12619904e+02, -4.71619453e+01, ...,
        -3.03358078e+00,  2.71057296e+00,  7.67189503e+00],
       [-3.44714233e+02,  1.26758141e+02, -5.61771698e+01, ...,
        -7.80225849e+00, -1.77907360e+00,  5.83541918e+00],
       [-3.15933838e+02,  9.56758881e+01, -3.80477676e+01, ...,
         5.77580512e-01, -1.14637651e+01, -5.96535349e+00]], dtype=float32)

In [ ]:
data_class
# Sound selction feature, "optimal feature selection" PCA LDA dimention deduction
# WIKI Dimentionality reduction

array(['bark', 'local_sound', 'local_sound', ..., 'horn', 'horn', 'horn'],
      dtype='<U12')

In [ ]:
data_class=np.array(pd.get_dummies(data_class))
data_class.shape

(8732, 6)

In [ ]:
data_feature_train,data_feature_test,data_class_train,data_class_test=train_test_split(data_feature,data_class,test_size=0.2,random_state=0)

In [ ]:
data_feature_train

array([[-1.3183614e+02,  1.1397464e+02, -2.3956861e+01, ...,
         3.3314774e+00, -1.4786111e+00,  2.8736601e+00],
       [-1.4074220e+01,  9.1916939e+01, -8.6787205e+00, ...,
        -3.3844025e+00, -5.2119040e+00, -1.5936139e+00],
       [-4.9532028e+01,  1.5521857e-01, -2.0369110e+01, ...,
         2.0491767e+00, -8.0537486e-01,  2.7793026e+00],
       ...,
       [-4.2699329e+02,  9.2890648e+01,  3.0233369e+00, ...,
         8.6335993e-01,  6.4766800e-01,  7.8490508e-01],
       [-1.4607024e+02,  1.3709459e+02, -3.4298344e+01, ...,
         1.3777871e+00, -1.9530842e+00, -8.9652115e-01],
       [-4.2167450e+02,  2.1169032e+02,  2.6820304e+00, ...,
        -5.1484952e+00, -3.6400859e+00, -1.3321606e+00]], dtype=float32)

In [ ]:
data_feature_train.shape

(6985, 40)

In [ ]:
data_feature_test.shape

(1747, 40)

In [ ]:
data_class_train.shape

(6985, 6)

In [ ]:
data_class_test.shape

(1747, 6)

In [ ]:
print(tensorflow.__version__)

2.8.0


In [ ]:
audio_data_labels=data_class.shape[1]
audio_data_labels

6

Creating a deep learning model

In [ ]:
"""
This CNN model has four layers
"""
model=Sequential()
# The First Layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# The Second Layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#The Third Layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# The Final Layer
model.add(Dense(audio_data_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4100      
                                                                 
 activation_4 (Activation)   (None, 100)               0         
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 200)               20200     
                                                                 
 activation_5 (Activation)   (None, 200)               0         
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 100)              

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
"""
Here I will use the above model.
"""
epochs_amount = 100
batch_amount = 32
check_pointer = ModelCheckpoint(filepath='model_audio.hdf5',verbose=1, save_best_only=True)
start = datetime.now()
model.fit(data_feature_train, data_class_train, batch_size=batch_amount, epochs=epochs_amount, validation_data=(data_feature_test, data_class_test), callbacks=[check_pointer], verbose=1)
duration = datetime.now() - start
print("Total training time: ", duration)

In [ ]:
"""
I put this here to show the accuracy of the model
"""
print(model.evaluate(data_feature_test,data_class_test,verbose=0)[1])

0.8465941548347473


Testing My Model

In [ ]:
"""
Here I am predicting the shape and feature of the audio data.
"""
audio_data="/content/drive/MyDrive/BigDataProject1/Sounds8K/bark.wav"
prediction_feature=wav_features(audio_data)
prediction_feature=prediction_feature.reshape(1,-1)
np.argmax(model.predict(prediction_feature), axis=-1)

array([3])

In [ ]:
prediction_feature.shape

(1, 40)

In [ ]:
data_feature_test[1]

array([-466.1843    ,    1.5388287 ,  -34.397358  ,   35.715336  ,
        -15.16693   ,  -18.850813  ,   -0.7415814 ,  -15.999888  ,
        -21.354332  ,    7.650683  ,  -29.03145   ,  -19.142824  ,
         -2.6798913 ,   -8.466883  ,  -14.7660475 ,   -7.004778  ,
         -7.103754  ,    8.887754  ,   14.911873  ,   21.47102   ,
         21.336626  ,    0.9169517 ,  -18.795404  ,   -5.001721  ,
         -0.70152706,    2.9139898 ,   -6.710599  ,  -16.638536  ,
         -9.821647  ,   12.8619585 ,    0.6552978 ,  -23.953394  ,
        -15.200551  ,    9.210791  ,   10.419799  ,   -0.57916135,
         -1.2440345 ,   17.722294  ,   13.837573  ,   -5.164349  ],
      dtype=float32)

In [ ]:
np.argmax(data_feature_test,axis=1)

array([1, 3, 3, ..., 1, 1, 1])

In [ ]:
audio_data="/content/drive/MyDrive/BigDataProject1/Sounds8K/AirRaidSirens.mp3"
# audio_data="content/Sounds8K/bark.wav"
audio, sample_rate = librosa.load(audio_data, res_type='kaiser_fast')
audio_data_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
audio_data_scaled_features = np.mean(audio_data_features.T,axis=0)

print(audio_data_scaled_features)
audio_data_scaled_features=audio_data_scaled_features.reshape(1,-1)
print(audio_data_scaled_features)
print(audio_data_scaled_features.shape)

predicted_audio_data_label=model.predict(audio_data_scaled_features)
prediction_of_the_audio_class=np.argmax(predicted_audio_data_label,axis=1)
prediction_of_the_audio_class

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[-3.0927805e+02  1.3172385e+02 -5.4930008e+01  1.6581200e+01
 -2.1007229e+01 -1.6556790e+00 -1.2015638e+01  6.9499283e+00
 -6.3267832e+00  3.0090704e+00 -3.5073764e+00  2.7442539e+00
 -2.4538140e+00 -4.2226110e+00  1.7021154e+00  1.6267620e+01
  1.3523576e+01  4.6997609e+00 -1.9422261e+00 -1.3986921e+00
  1.5475668e+00  5.3678365e+00  8.6449738e+00  4.1029415e+00
 -2.8851612e+00  1.0652226e+00  6.1513014e+00 -1.3851628e-02
 -2.3089600e+00  5.0834525e-01  9.6132421e+00  9.1986179e+00
  5.2164950e+00 -7.9539919e-01 -1.3237062e+00 -3.1928692e+00
 -3.9449828e+00  1.5759032e+00  4.3784671e+00 -4.2267990e+00]
[[-3.0927805e+02  1.3172385e+02 -5.4930008e+01  1.6581200e+01
  -2.1007229e+01 -1.6556790e+00 -1.2015638e+01  6.9499283e+00
  -6.3267832e+00  3.0090704e+00 -3.5073764e+00  2.7442539e+00
  -2.4538140e+00 -4.2226110e+00  1.7021154e+00  1.6267620e+01
   1.3523576e+01  4.6997609e+00 -1.9422261e+00 -1.3986921e+00
   1.5475668e+00  5.3678365e+00  8.6449738e+00  4.1029415e+00
  -2.8851612e+00 

array([4])

In [ ]:
d_set = data_set['class'].unique()
print(d_set[prediction_of_the_audio_class])
print(prediction_of_the_audio_class)
data_set['class'].unique()

['siren']
[4]


array(['bark', 'local_sound', 'horn', 'gun', 'siren', 'construction'],
      dtype=object)

In [ ]:
import smtplib, ssl
alert_s = d_set[prediction_of_the_audio_class]

In [ ]:
if alert_s == 'siren' or alert_s == 'gun':
    print("Emergency")
    port = 587  
    smtp_server = "smtp.gmail.com"
    sender_email = "my@gmail.com"
    receiver_email = "your@gmail.com"
    password = "Password"
    message = """\
    Subject: Emergency Alert Protocol
    This message is sent to inform you that an.
    Emergency sound has been detected."""

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls(context=context)
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
else:
    print("Everything is Fine")

Emergency


SMTPAuthenticationError: ignored